In [1]:
import sys
import os
os.chdir("/home/rana-helal/PycharmProjects/fake_news_classifier")

from src.data_loader import load_kaggle_data

df_kaggle = load_kaggle_data()


# preprocessing


In [11]:
#!python -m spacy download en_core_web_sm


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [13]:
from tqdm import tqdm
tqdm.pandas()


In [14]:
from src.preprocessing import remove_duplicates_and_missing, clean_for_ml

# Apply text cleaning for ML models

df_cleaned = remove_duplicates_and_missing(df_kaggle)


In [16]:

df_cleaned['clean_text'] = df_cleaned['text'].progress_apply(clean_for_ml)


100%|██████████| 44054/44054 [24:07<00:00, 30.44it/s]  


In [17]:
from src.utils import save_cleaned_data

save_cleaned_data(df_cleaned, "kaggle_clean_ml.csv")

 Saved cleaned data to data/processed/kaggle_clean_ml.csv


# label encoding

In [2]:
import pandas as pd

df = pd.read_csv("data/processed/kaggle_clean_ml.csv")
df.head()


,title,text,subject,label,clean_text
0,Trump Gets HUMILIATED By His Own Joint Chiefs...,It appears Donald Trump jumped the gun when he...,News,fake,appear donald trump jump gun announce ban tran...
1,Myanmar bars U.N. rights investigator before v...,GENEVA (Reuters) - The U.N. independent invest...,worldnews,real,geneva reuters un independent investigator hum...
2,UNREAL! SHEILA JACKSON LEE DEMANDS THAT TRUMP ...,Neil Cavuto faced off with Congresswoman Sheil...,politics,fake,neil cavuto face congresswoman sheila jackson ...
3,White House's Cohn says 'fair trade' means rec...,WASHINGTON (Reuters) - The Trump administratio...,politicsNews,real,washington reuters trump administration want t...
4,WOW! WIKILEAKS Emails Shows How Hillary Will B...,Clinton s radicalized rhetoric has championed...,politics,fake,clinton s radicalize rhetoric champion ban cal...


In [3]:
df['label'] = df['label'].map({'fake': 0, 'real': 1})

In [4]:
df['label'].value_counts()


label
0    22847
1    21207
Name: count, dtype: int64

# Prepare TF-IDF vectorization

In [10]:
texts = df["clean_text"].astype(str)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
df = df.dropna(subset=["clean_text"])

tfidf = TfidfVectorizer(max_features=10000)
X_tfidf = tfidf.fit_transform(df["clean_text"])


In [12]:
from src.utils import save_pickle
save_pickle(tfidf, "models/tfidf_vectorizer.pkl")


tokenizer saved to models/tfidf_vectorizer.pkl



# split the data

In [8]:
from src.utils import load_pickle

tfidf = load_pickle("models/tfidf_vectorizer.pkl")


Loading tokenizer from models/tfidf_vectorizer.pkl


In [13]:
from src.utils import split_data

y = df['label']

X_train, X_val, X_test, y_train, y_val, y_test = split_data(X_tfidf, y)

Data split into train, validation, and test sets:


# train the model

In [14]:
from src.models.logistic_model import train_logistic_model

model = train_logistic_model(X_train, y_train, X_val, y_val)


Confusion matrix saved to visualizations/confusion_matrices/logistic_cm.png
[INFO] Logistic Regression model trained and saved as 'logistic_model.h5'.


In [15]:
from sklearn.metrics import accuracy_score

y_pred_test = model.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_test.round())}")

Test Accuracy: 0.9851034796452126
